<a href='https://ai.meng.duke.edu'> = <img align="left" style="padding-top:10px;" src=https://storage.googleapis.com/aipi_datasets/Duke-AIPI-Logo.png>

# Neural Nets for Neural Images: Assessing Brain Age with Computer Vision
### Contributors: Dr. Jeffrey Petrella, Maria Williams

This project uses transfer learning to train a ResNet18 model to identify the age of a brain from its MRI scan. This kind of identification could potentially be used to flag various congnitive disorders such as Alzhiemers Disease if the "brain age" is identified as more advanced than the patient's chronological age.

In [1]:
import os
from google.colab import drive
from google.colab import files

#for image processing
!pip install pillow
import random
import numpy as np
import tensorflow as tf
from PIL import Image
from matplotlib import pyplot as plt

#for metadata
import pandas as pd

#for data file structure
!pip install pip install split-folders
import splitfolders
import shutil

#for modeling
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, ToTensor, Resize
from torch.utils.data import Subset, DataLoader, Dataset
from sklearn.model_selection import train_test_split
from torchsummary import summary

#for metrics
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Step 1: Link Notebook to GitHub and Google Drive

In [2]:
# Remove Colab default sample_data
!rm -r ./sample_data

# Clone GitHub files to colab workspace
repo_name = "BrainAgingComputerVision" # Enter repo name
git_path = 'https://github.com/MRWilliamsGit/BrainAgingComputerVision.git'
!git clone "{git_path}"

#!pip install -r "{os.path.join(repo_name,'requirements.txt')}"

# Connect to Google Drive
#drive.mount('/content/gdrive')
#%cd gdrive/MyDrive/School

Cloning into 'BrainAgingComputerVision'...
remote: Enumerating objects: 170, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 170 (delta 22), reused 60 (delta 22), pack-reused 110
Receiving objects: 100% (170/170), 164.50 MiB | 33.48 MiB/s, done.
Resolving deltas: 100% (68/68), done.


### Step 2: call main

In [3]:
!python3 /content/BrainAgingComputerVision/main.py

Hey There
